<a href="https://colab.research.google.com/github/capGoblin/Stock-Recommendation/blob/main/Stock_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

In [ ]:
!pip install ordered-set

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.4 MB/s eta 0:00:00


In [ ]:
!wget http://www-nlp.stanford.edu/data/glove.840B.300d.zip
!unzip glove.840B.300d.zip


--2023-07-19 08:33:01--  http://www-nlp.stanford.edu/data/glove.840B.300d.zip
Resolving www-nlp.stanford.edu (www-nlp.stanford.edu)... 171.64.67.140
Connecting to www-nlp.stanford.edu (www-nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2023-07-19 08:34:06--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2023-07-19 08:34:06--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTT

In [ ]:
import numpy as np
import pandas as pd
import re
import yfinance as yf
from datetime import datetime, timedelta
import datetime
import string
from nltk.corpus import stopwords
import contractions
import ast

import nltk

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d gpreda/reddit-wallstreetsbets-posts

 67% 11.0M/16.5M [00:00<00:00, 96.8MB/s]
100% 16.5M/16.5M [00:00<00:00, 115MB/s] 


In [ ]:
!unzip /content/reddit-wallstreetsbets-posts.zip

Archive:  /content/reddit-wallstreetsbets-posts.zip
  inflating: reddit_wsb.csv          


In [ ]:
!wget http://www-nlp.stanford.edu/data/glove.840B.300d.zip


--2023-07-19 07:14:24--  http://www-nlp.stanford.edu/data/glove.840B.300d.zip
Resolving www-nlp.stanford.edu (www-nlp.stanford.edu)... 171.64.67.140
Connecting to www-nlp.stanford.edu (www-nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2023-07-19 07:14:24--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2023-07-19 07:14:24--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTT

In [ ]:
!unzip glove.840B.300d.zip

Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


In [ ]:
data1 = pd.read_csv('/content/reddit_wsb.csv')
data2 = pd.read_csv('/content/100days.csv')


In [ ]:
pd.DataFrame(data1)

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28 21:32:10
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28 21:28:57
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28 21:26:56
...,...,...,...,...,...,...,...,...
53182,What I Learned Investigating SAVA FUD Spreaders,238,owd2pn,https://www.reddit.com/r/wallstreetbets/commen...,87,1.627906e+09,***TLDR: Three bitter scientists partnered up ...,2021-08-02 15:03:27
53183,"Daily Popular Tickers Thread for August 02, 20...",228,owd1a5,https://www.reddit.com/r/wallstreetbets/commen...,1070,1.627906e+09,\nYour daily hype thread. Please keep the shit...,2021-08-02 15:01:03
53184,Hitler reacts to the market being irrational,7398,owc5dr,https://v.redd.it/46jxu074exe71,372,1.627902e+09,NaN,2021-08-02 13:59:35
53185,"Daily Discussion Thread for August 02, 2021",338,owbfjf,https://www.reddit.com/r/wallstreetbets/commen...,11688,1.627898e+09,Your daily trading discussion thread. Please k...,2021-08-02 13:00:16


In [ ]:
pd.DataFrame(data2)

,id,ticker,growth,title,flair,score,upvote_ratio,author,num_comments,text,created,url,timestamp
0,krsa5c,None,NaN,update $ yolo,YOLO,15,1.00,dhsmatt2,290,all i am still holding the same major position...,2021-01-06,https://www.reddit.com/r/wallstreetbets/commen...,1609909200
1,krxpdq,None,NaN,upcoming nio day partnership rumors,Discussion,9,1.00,Fancy-Blackberry-332,60,nio day is coming up this saturday and it has ...,2021-01-06,https://www.reddit.com/r/wallstreetbets/commen...,1609909200
2,ks2don,None,NaN,call me a retard if you want jmia 🚀,YOLO,5,0.86,JKK201519,42,yeah thats fucking right ladies and gents ahem...,2021-01-06,https://www.reddit.com/r/wallstreetbets/commen...,1609909200
3,ks3lno,None,NaN,cannabis and tendies 🚀,Discussion,8,1.00,drtywlf,40,my fellow autists lend me your ears i come bef...,2021-01-06,https://www.reddit.com/r/wallstreetbets/commen...,1609909200
4,kru2jl,None,NaN,pltr the path forward🚀🚀🚀🚀🚀🚀🚀🚀,Discussion,6,0.88,MC_lgnd,46,remember that tomorrow jan at am et papa karp ...,2021-01-06,https://www.reddit.com/r/wallstreetbets/commen...,1609909200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3275,mk2651,PLTR,4.20%,pltr multiple news articles last weekend pltr ...,Discussion,34,0.80,Opposite-Golf-3232,246,pltr palantir technologies credit suisse morga...,2021-04-04,https://www.reddit.com/r/wallstreetbets/commen...,1617508800
3276,mk0gae,VIAC,-6.18%,heres why viac is setting up for a strong rebound,DD,10,0.68,universeatom,306,viacomcbs viac is down in nearly two weeks and...,2021-04-04,https://www.reddit.com/r/wallstreetbets/commen...,1617508800
3277,mk4pod,PAVE,0.48%,an investment that will benefit from the infra...,Discussion,30,0.86,ralphsdiorama,40,if you are looking for an infrastructure inves...,2021-04-04,https://www.reddit.com/r/wallstreetbets/commen...,1617508800
3278,mk5h3m,None,NaN,the us govt has a huge incentive to massively ...,Discussion,171,0.93,Natural_Profession_8,830,tldr the us federal government has a massive i...,2021-04-04,https://www.reddit.com/r/wallstreetbets/commen...,1617508800


In [ ]:
idd = data1[data1['body'] == 'NaN'].index
yaa = data1[data1['body'] == 'NaN'].index

removed_nan = data1.dropna(subset=['body'])

In [ ]:
arragencol = removed_nan[['id', 'title', 'score', 'comms_num', 'body', 'timestamp', 'url', 'created']]

In [ ]:
arragencol = arragencol.rename(columns={'timestamp': 'created', 'created': 'timestamp'})

In [ ]:
arragencol = arragencol.rename(columns={'body': 'text', 'comms_num': 'num_comments'})

In [ ]:
arragencol

,id,title,score,num_comments,text,created,url,timestamp
2,l6uhhn,Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,https://www.reddit.com/r/wallstreetbets/commen...,1.611862e+09
6,l6uf6d,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,https://www.reddit.com/r/wallstreetbets/commen...,1.611862e+09
7,l6ub9l,THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,https://www.reddit.com/r/wallstreetbets/commen...,1.611862e+09
10,l6uao1,"We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,https://www.reddit.com/r/wallstreetbets/commen...,1.611862e+09
12,l6u9wu,"Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,https://www.reddit.com/r/wallstreetbets/commen...,1.611861e+09
...,...,...,...,...,...,...,...,...
53181,owfbxp,Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but...",2021-08-02 17:11:36,https://www.reddit.com/r/wallstreetbets/commen...,1.627913e+09
53182,owd2pn,What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...,2021-08-02 15:03:27,https://www.reddit.com/r/wallstreetbets/commen...,1.627906e+09
53183,owd1a5,"Daily Popular Tickers Thread for August 02, 20...",228,1070,\nYour daily hype thread. Please keep the shit...,2021-08-02 15:01:03,https://www.reddit.com/r/wallstreetbets/commen...,1.627906e+09
53185,owbfjf,"Daily Discussion Thread for August 02, 2021",338,11688,Your daily trading discussion thread. Please k...,2021-08-02 13:00:16,https://www.reddit.com/r/wallstreetbets/commen...,1.627898e+09


In [ ]:
arragencol['timestamp'] = arragencol['timestamp'].apply(lambda x: '{:.0f}'.format(x))


In [ ]:
arragencol

,id,title,score,num_comments,text,created,url,timestamp
2,l6uhhn,Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,https://www.reddit.com/r/wallstreetbets/commen...,1611862235
6,l6uf6d,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,https://www.reddit.com/r/wallstreetbets/commen...,1611861987
7,l6ub9l,THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,https://www.reddit.com/r/wallstreetbets/commen...,1611861571
10,l6uao1,"We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,https://www.reddit.com/r/wallstreetbets/commen...,1611861505
12,l6u9wu,"Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,https://www.reddit.com/r/wallstreetbets/commen...,1611861430
...,...,...,...,...,...,...,...,...
53181,owfbxp,Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but...",2021-08-02 17:11:36,https://www.reddit.com/r/wallstreetbets/commen...,1627913496
53182,owd2pn,What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...,2021-08-02 15:03:27,https://www.reddit.com/r/wallstreetbets/commen...,1627905807
53183,owd1a5,"Daily Popular Tickers Thread for August 02, 20...",228,1070,\nYour daily hype thread. Please keep the shit...,2021-08-02 15:01:03,https://www.reddit.com/r/wallstreetbets/commen...,1627905663
53185,owbfjf,"Daily Discussion Thread for August 02, 2021",338,11688,Your daily trading discussion thread. Please k...,2021-08-02 13:00:16,https://www.reddit.com/r/wallstreetbets/commen...,1627898416


# Data Cleaning

In [ ]:
blacklist_words = ['TL;DR', 'DD', 'AM', 'ELON', 'ICE', 'USA', 'FEB', 'SEPT', 'PRAY', 'DJIA', 'POS', 'MACD', 'ATH', 'CEO', 'AH', 'DR', 'NOT', 'GOD', 'ROFL', 'KYS', 'DOW', 'GDP', 'CFO', 'PT', 'OK', 'GG', 'AT', 'GOAT', 'OP', 'YOLO', 'RISE', 'PR', 'IN', 'USD', 'ITM', 'TL', 'OTM', 'TYS', 'EDIT', 'LPT', 'MILF', 'EZ', 'ARK', 'PS', 'SEP', 'IPO', 'IS', 'DEC', 'APES', 'TICK', 'HQ', 'RH', 'SEC', 'LMAO', 'AUG', 'THIS', 'JAN', 'TOS', 'NOV', 'BABY', 'IV', 'CTO', 'CPU', 'MY', 'ER', 'ATM', 'DOJ', 'LGMA', 'IMO', 'WTF', 'LOL', 'URL', 'COD', 'WE', 'FOMO', 'MOST', 'BEZOS', 'PDFUA', 'BE', 'RED', 'IPA', 'SSN', 'US', 'TO', 'OCT', 'FDA', 'IT', 'RIP', 'FBI', 'FL', 'ISIS', 'BMW', 'PM', 'I', 'AYYMD', 'FIFA', 'RAW', 'IL', 'S', 'WSB', 'BTFD', 'LMFAO', 'CA', 'AND', 'A', 'STILL', 'JUL', 'BUY', 'BUT', 'THE', 'FD', 'PC']

In [ ]:

def extract_tickers(text):
    pattern1 = r'\$([A-Z]{1,6})\b'
    pattern2 = r'\b([A-Z]{1,6})\b'

    tickers = re.findall(pattern1, text) + re.findall(pattern2, text)

    valid_tickers = [ticker for ticker in tickers if len(ticker) > 1 and ticker not in blacklist_words]
    valid_tickers = list(set(valid_tickers))

    return valid_tickers

# Assuming 'df' is your DataFrame and 'title' and 'body' are the columns containing the post title and body text
arragencol['title_tickers'] = arragencol['title'].apply(lambda x: extract_tickers(x))
arragencol['body_tickers'] = arragencol['text'].apply(lambda x: extract_tickers(x))


In [ ]:
arragencol

,id,title,score,num_comments,text,created,url,timestamp,title_tickers,body_tickers
2,l6uhhn,Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,https://www.reddit.com/r/wallstreetbets/commen...,1611862235,[],"[NASDAQ, GME]"
6,l6uf6d,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,https://www.reddit.com/r/wallstreetbets/commen...,1611861987,"[STOCK, DATE, SHORT, HAVE, DOESN, AN]","[GME, PANIC, DON, STOCK, HAPPEN, FRIDAY, THAT,..."
7,l6ub9l,THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,https://www.reddit.com/r/wallstreetbets/commen...,1611861571,[MOMENT],"[GME, THEM, BB, THEY, SCARED, ARE]"
10,l6uao1,"We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,https://www.reddit.com/r/wallstreetbets/commen...,1611861505,[],"[GME, AMC]"
12,l6u9wu,"Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,https://www.reddit.com/r/wallstreetbets/commen...,1611861430,"[GME, AG, SLV]","[OUT, GME, OWN, AG, SLV, FAIR, JPM, SHORTS]"
...,...,...,...,...,...,...,...,...,...,...
53181,owfbxp,Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but...",2021-08-02 17:11:36,https://www.reddit.com/r/wallstreetbets/commen...,1627913496,[TSLA],[FSD]
53182,owd2pn,What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...,2021-08-02 15:03:27,https://www.reddit.com/r/wallstreetbets/commen...,1627905807,"[SAVA, FUD]","[SAVA, OF, COG, UK, STAT, ROB, TLDR, ADAS, HOW..."
53183,owd1a5,"Daily Popular Tickers Thread for August 02, 20...",228,1070,\nYour daily hype thread. Please keep the shit...,2021-08-02 15:01:03,https://www.reddit.com/r/wallstreetbets/commen...,1627905663,"[GME, BABA, AMD]",[]
53185,owbfjf,"Daily Discussion Thread for August 02, 2021",338,11688,Your daily trading discussion thread. Please k...,2021-08-02 13:00:16,https://www.reddit.com/r/wallstreetbets/commen...,1627898416,[],[]


In [ ]:
arragencol['ticker'] = arragencol['title_tickers'] + arragencol['body_tickers']

# Drop the 'text_tickers' and 'title_tickers' columns
arragencol = arragencol.drop(['title_tickers', 'body_tickers'], axis=1)

# Get unique values in the 'combined_tickers' column
unique_tickers = arragencol['ticker'].apply(lambda x: set(x))

In [ ]:
arragencol

,id,title,score,num_comments,text,created,url,timestamp,ticker
2,l6uhhn,Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,https://www.reddit.com/r/wallstreetbets/commen...,1611862235,"[NASDAQ, GME]"
6,l6uf6d,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,https://www.reddit.com/r/wallstreetbets/commen...,1611861987,"[STOCK, DATE, SHORT, HAVE, DOESN, AN, GME, PAN..."
7,l6ub9l,THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,https://www.reddit.com/r/wallstreetbets/commen...,1611861571,"[MOMENT, GME, THEM, BB, THEY, SCARED, ARE]"
10,l6uao1,"We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,https://www.reddit.com/r/wallstreetbets/commen...,1611861505,"[GME, AMC]"
12,l6u9wu,"Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,https://www.reddit.com/r/wallstreetbets/commen...,1611861430,"[GME, AG, SLV, OUT, GME, OWN, AG, SLV, FAIR, J..."
...,...,...,...,...,...,...,...,...,...
53181,owfbxp,Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but...",2021-08-02 17:11:36,https://www.reddit.com/r/wallstreetbets/commen...,1627913496,"[TSLA, FSD]"
53182,owd2pn,What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...,2021-08-02 15:03:27,https://www.reddit.com/r/wallstreetbets/commen...,1627905807,"[SAVA, FUD, SAVA, OF, COG, UK, STAT, ROB, TLDR..."
53183,owd1a5,"Daily Popular Tickers Thread for August 02, 20...",228,1070,\nYour daily hype thread. Please keep the shit...,2021-08-02 15:01:03,https://www.reddit.com/r/wallstreetbets/commen...,1627905663,"[GME, BABA, AMD]"
53185,owbfjf,"Daily Discussion Thread for August 02, 2021",338,11688,Your daily trading discussion thread. Please k...,2021-08-02 13:00:16,https://www.reddit.com/r/wallstreetbets/commen...,1627898416,[]


In [ ]:
arragencol = arragencol[['id','ticker', 'title', 'score', 'num_comments', 'text', 'created', 'url', 'timestamp']]

In [ ]:
arragencol


,id,ticker,title,score,num_comments,text,created,url,timestamp
2,l6uhhn,"[NASDAQ, GME]",Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,https://www.reddit.com/r/wallstreetbets/commen...,1611862235
6,l6uf6d,"[STOCK, DATE, SHORT, HAVE, DOESN, AN, GME, PAN...",SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,https://www.reddit.com/r/wallstreetbets/commen...,1611861987
7,l6ub9l,"[MOMENT, GME, THEM, BB, THEY, SCARED, ARE]",THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,https://www.reddit.com/r/wallstreetbets/commen...,1611861571
10,l6uao1,"[GME, AMC]","We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,https://www.reddit.com/r/wallstreetbets/commen...,1611861505
12,l6u9wu,"[GME, AG, SLV, OUT, GME, OWN, AG, SLV, FAIR, J...","Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,https://www.reddit.com/r/wallstreetbets/commen...,1611861430
...,...,...,...,...,...,...,...,...,...
53181,owfbxp,"[TSLA, FSD]",Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but...",2021-08-02 17:11:36,https://www.reddit.com/r/wallstreetbets/commen...,1627913496
53182,owd2pn,"[SAVA, FUD, SAVA, OF, COG, UK, STAT, ROB, TLDR...",What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...,2021-08-02 15:03:27,https://www.reddit.com/r/wallstreetbets/commen...,1627905807
53183,owd1a5,"[GME, BABA, AMD]","Daily Popular Tickers Thread for August 02, 20...",228,1070,\nYour daily hype thread. Please keep the shit...,2021-08-02 15:01:03,https://www.reddit.com/r/wallstreetbets/commen...,1627905663
53185,owbfjf,[],"Daily Discussion Thread for August 02, 2021",338,11688,Your daily trading discussion thread. Please k...,2021-08-02 13:00:16,https://www.reddit.com/r/wallstreetbets/commen...,1627898416


In [ ]:
import datetime
import yfinance as yf

# Function to check if a given date is a trading day
def is_trading_day(date):
    return date.weekday() < 5  # Monday to Friday (0 to 4)

# Function to find the next trading day
def find_next_trading_day(date):
    while not is_trading_day(date):
        date += datetime.timedelta(days=1)
    return date

# Modified growth function using NumPy and pandas
def growth(tickers, created):
    if len(tickers) == 0:
        return []

    growth_percentages = []

    for ticker in tickers:
        if not ticker or ticker == "None":
            growth_percentages.append("N/A")
            continue

        try:
            ticker_date_start = find_next_trading_day(pd.to_datetime(created))

            ticker_date_end = ticker_date_start + pd.DateOffset(days=0.8)
            ticker_date_end = find_next_trading_day(ticker_date_end)

            ticker_data = yf.download(ticker, start=ticker_date_start, end=ticker_date_end)

            if ticker_data.empty:
                growth_percentages.append("No Price Data1")
                continue

            price_initial = ticker_data['Close'].iloc[0]

            ticker_date_end += pd.DateOffset(days=90)
            ticker_date_end = find_next_trading_day(ticker_date_end)

            ticker_data = yf.download(ticker, start=ticker_date_start, end=ticker_date_end)

            if ticker_data.empty:
                growth_percentages.append("No Price Data")
                continue

            price_final = ticker_data['Close'].iloc[-1]

            percentage = ((price_final / price_initial) - 1) * 100
            growth_percentages.append("{:.2f}%".format(percentage))

        except (ValueError, IndexError, KeyError):
            growth_percentages.append("Invalid Ticker or Date")

    return growth_percentages

# Example usage
date = '2022-09-18 01:30:35'  # Example created date
tickers = np.array(["TSLA", "GME", "AAPL", "NOKIA"])

growth_percentage = growth(tickers, date)
print(growth_percentage)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['NOKIA']: Exception('%ticker%: No timezone found, symbol may be delisted')



['-51.51%', '-31.22%', '-14.31%', 'No Price Data1']


In [ ]:
import pandas as pd
from multiprocessing import Pool

# Function to calculate growth for a single row
def calculate_growth(row):
    index, data = row
    ticker = data['ticker']
    created = data['created']
    return growth(ticker, created)

# Modified growth function with parallel processing
num_processes = 2
def calculate_growth_parallel(df):
    pool = Pool(processes=num_processes)  # Create a process pool
    results = []

    for i, row in enumerate(df.iterrows()):
        result = calculate_growth(row)
        results.append(result)

        if (i + 1) % 250 == 0:
            print("Processed", i + 1, "rows")

    pool.close()
    pool.join()
    return results

arragencol_subset = arragencol[5000:10000]
arragencol_subset['Growth Percentage'] = calculate_growth_parallel(arragencol_subset)

# Print the updated DataFrame
print(arragencol_subset)


In [ ]:
arragencol_subset

In [ ]:
arragencol_subset_concat = pd.read_csv('/content/arragencol_subset_concat.csv')

In [ ]:
arragencol_subset_concat = arragencol_subset_concat.rename(columns={'Growth Percentage': 'growth'})
arragencol_subset_concat = arragencol_subset_concat.drop('timestamp', axis=1)
arragencol_subset_concat = arragencol_subset_concat.drop('url', axis=1)
arragencol_subset_concat = arragencol_subset_concat.drop('created', axis=1)

arragencol_subset_concat = arragencol_subset_concat[['id','ticker', 'growth', 'title', 'score', 'num_comments', 'text']]

In [ ]:
data2 = data2.drop('timestamp', axis=1)
data2 = data2.drop('created', axis=1)

data2 = data2.drop('url', axis=1)
data2 = data2.drop('author', axis=1)
data2 = data2.drop('flair', axis=1)
data2 = data2.drop('upvote_ratio', axis=1)

data2 = data2.dropna(subset='growth')

In [ ]:
data2

,id,ticker,growth,title,score,num_comments,text
8,krx451,GME,811.69%,k gme sharesoptions lets get it gme gay fam,14,42,sharesoptions k
11,ks1tzw,GME,811.69%,gme🚀🚀🚀,28,37,all right all of you autists and filthy casual...
14,ksjhuu,MGNI,47.13%,$mgni 🌈🐻 have created a wonderful buying oppor...,7,97,tldr $ pt positions $c 🚀🚀🚀🚀🚀 good afternoon re...
19,kt79b2,GME,775.88%,$gme gang ryan cohen could be attending the ic...,9,164,we might be hearing for the first time directl...
23,kt9enf,GME,775.88%,gme newbies dont lose hope this price action i...,14,8,bears feast on low volume hence why gme drops ...
...,...,...,...,...,...,...,...
3268,mfc86c,AMRS,-14.75%,$amrs 🚀 🌕 the ultimate dd 🚀 🌕 deep longterm va...,20,630,note nothing in this post is financial advice ...
3275,mk2651,PLTR,4.20%,pltr multiple news articles last weekend pltr ...,34,246,pltr palantir technologies credit suisse morga...
3276,mk0gae,VIAC,-6.18%,heres why viac is setting up for a strong rebound,10,306,viacomcbs viac is down in nearly two weeks and...
3277,mk4pod,PAVE,0.48%,an investment that will benefit from the infra...,30,40,if you are looking for an infrastructure inves...


In [ ]:
arragencol_subset_concat_copy = arragencol_subset_concat[:]


In [ ]:
arragencol_subset_concat_copy

,id,ticker,growth,title,score,num_comments,text
0,l6uhhn,['GME' 'NASDAQ'],"['-45.79%', 'No Price Data1']",Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...
1,l6uf6d,['HAVE' 'SHORT' 'AN' 'DATE' 'STOCK' 'DOESN' 'D...,"['No Price Data1', 'No Price Data1', '44.91%',...",SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...
2,l6ub9l,['MOMENT' 'ARE' 'BB' 'THEY' 'THEM' 'SCARED' 'G...,"['No Price Data1', '7.52%', '-35.74%', 'No Pri...",THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...
3,l6uao1,['AMC' 'GME'],"['-23.08%', '-45.79%']","We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...
4,l6u9wu,['AG' 'SLV' 'GME' 'OUT' 'FAIR' 'OWN' 'SHORTS' ...,"['-14.24%', '-2.96%', '-45.79%', '34.61%', '2....","Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...
...,...,...,...,...,...,...,...
24733,owfbxp,"['TSLA', 'FSD']","['70.30%', '0.13%']",Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but..."
24734,owd2pn,"['SAVA', 'FUD', 'SAVA', 'HOWARD', 'STAT', 'ADA...","['-42.35%', 'No Price Data1', '-42.35%', 'No P...",What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...
24735,owd1a5,"['BABA', 'GME', 'AMD']","['-14.95%', '26.92%', '15.28%']","Daily Popular Tickers Thread for August 02, 20...",228,1070,\nYour daily hype thread. Please keep the shit...
24736,owbfjf,[],[],"Daily Discussion Thread for August 02, 2021",338,11688,Your daily trading discussion thread. Please k...


In [ ]:
def convert_string_to_list(string):
    string = string.strip("[]")  # Remove the brackets
    string = string.replace("'", "")

    if "," in string:
        # Type 2: ['TSLA', 'FSD']
        string = string.replace(" ", "")
        string_list = [item.strip(",") for item in string.split(",") if item.strip(",")]
    else:
        # Type 1: ['GME', 'NASDAQ']
        string_list = [item.strip() for item in string.split()]

    return string_list

def get_index(row):
    ticker_str = row['ticker']
    growth_str = row['growth']

    if isinstance(growth_str, str):
        ticker_arr = convert_string_to_list(ticker_str)
        growth_arr = ast.literal_eval(growth_str)

        indices_to_remove = []

        for i, (ticker, growth) in enumerate(zip(ticker_arr, growth_arr)):
            try:
                b = growth.rstrip('%')
                float_value = float(b)
            except ValueError:
                indices_to_remove.append(i)

        # Remove the corresponding items from ticker_arr and growth_arr
        ticker_arr = [ticker for i, ticker in enumerate(ticker_arr) if i not in indices_to_remove]
        growth_arr = [growth for i, growth in enumerate(growth_arr) if i not in indices_to_remove]

        # Update the ticker and growth columns of the row
        row['ticker'] = ticker_arr
        row['growth'] = growth_arr

    return row

# Apply the get_index function to each row in the DataFrame
for index, row in arragencol_subset_concat_copy.iterrows():
    arragencol_subset_concat_copy.loc[index] = get_index(row)

# Print the updated DataFrame
arragencol_subset_concat_copy


,id,ticker,growth,title,score,num_comments,text
0,l6uhhn,[GME],[-45.79%],Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...
1,l6uf6d,"[AN, DON, GME]","[44.91%, 20.81%, -45.79%]",SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...
2,l6ub9l,"[ARE, BB, GME]","[7.52%, -35.74%, -45.79%]",THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...
3,l6uao1,"[AMC, GME]","[-23.08%, -45.79%]","We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...
4,l6u9wu,"[AG, SLV, GME, OUT, FAIR, AG, SLV, GME, JPM]","[-14.24%, -2.96%, -45.79%, 34.61%, 2.34%, -14....","Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...
...,...,...,...,...,...,...,...
24733,owfbxp,"[TSLA, FSD]","[70.30%, 0.13%]",Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but..."
24734,owd2pn,"[SAVA, SAVA, UK]","[-42.35%, -42.35%, -15.25%]",What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...
24735,owd1a5,"[BABA, GME, AMD]","[-14.95%, 26.92%, 15.28%]","Daily Popular Tickers Thread for August 02, 20...",228,1070,\nYour daily hype thread. Please keep the shit...
24736,owbfjf,[],[],"Daily Discussion Thread for August 02, 2021",338,11688,Your daily trading discussion thread. Please k...


In [ ]:
from ordered_set import OrderedSet

arragencol_subset_concat_copy['ticker'] = arragencol_subset_concat_copy['ticker'].apply(lambda x: list(OrderedSet(x)))
arragencol_subset_concat_copy['growth'] = arragencol_subset_concat_copy['growth'].apply(lambda x: list(OrderedSet(x)))



In [ ]:
arragencol_subset_concat_copy_copy = arragencol_subset_concat_copy[:]

In [ ]:
def update_growth_and_ticker(row):
    ticker_list = row['ticker']
    growth_list = row['growth']

    if isinstance(growth_list, list) and len(growth_list) > 0:
        max_growth = max(growth_list)  # Get the maximum growth percentage
        max_index = growth_list.index(max_growth)  # Get the index of the maximum growth

        ticker_list = [ticker for i, ticker in enumerate(ticker_list) if i == max_index]
        growth_list = [growth for growth in growth_list if growth == max_growth]

    row['ticker'] = ticker_list
    row['growth'] = growth_list

    return row


# Apply the update_growth_and_ticker function to each row in the DataFrame
updated_df = arragencol_subset_concat_copy_copy.apply(update_growth_and_ticker, axis=1)

updated_df


,id,ticker,growth,title,score,num_comments,text
0,l6uhhn,[GME],[-45.79%],Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...
1,l6uf6d,[AN],[44.91%],SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...
2,l6ub9l,[ARE],[7.52%],THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...
3,l6uao1,[GME],[-45.79%],"We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...
4,l6u9wu,[OUT],[34.61%],"Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...
...,...,...,...,...,...,...,...
24733,owfbxp,[TSLA],[70.30%],Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but..."
24734,owd2pn,[SAVA],[-42.35%],What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...
24735,owd1a5,[GME],[26.92%],"Daily Popular Tickers Thread for August 02, 20...",228,1070,\nYour daily hype thread. Please keep the shit...
24736,owbfjf,[],[],"Daily Discussion Thread for August 02, 2021",338,11688,Your daily trading discussion thread. Please k...


In [ ]:
updated_df

,id,ticker,growth,title,score,num_comments,text
0,l6uhhn,[GME],[-45.79%],Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...
1,l6uf6d,[AN],[44.91%],SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...
2,l6ub9l,[ARE],[7.52%],THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...
3,l6uao1,[GME],[-45.79%],"We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...
4,l6u9wu,[OUT],[34.61%],"Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...
...,...,...,...,...,...,...,...
24733,owfbxp,[TSLA],[70.30%],Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but..."
24734,owd2pn,[SAVA],[-42.35%],What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...
24735,owd1a5,[GME],[26.92%],"Daily Popular Tickers Thread for August 02, 20...",228,1070,\nYour daily hype thread. Please keep the shit...
24736,owbfjf,[],[],"Daily Discussion Thread for August 02, 2021",338,11688,Your daily trading discussion thread. Please k...


In [ ]:
updated_df__ = updated_df[
    (updated_df['ticker'].str.len() > 0) &
    (updated_df['growth'].str.len() > 0)
]

updated_df__

,id,ticker,growth,title,score,num_comments,text
0,l6uhhn,[GME],[-45.79%],Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...
1,l6uf6d,[AN],[44.91%],SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...
2,l6ub9l,[ARE],[7.52%],THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...
3,l6uao1,[GME],[-45.79%],"We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...
4,l6u9wu,[OUT],[34.61%],"Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...
...,...,...,...,...,...,...,...
24731,owht70,[AMC],[5.31%],Downtrends ... How to deal with them.,28,36,"WSB, hi. You may know me from my previous shi..."
24732,owhamt,[CRSR],[-14.29%],"$CRSR August 20, 2021 Unusual Options Activity 👀",200,115,"A)50$ strike\n3,048 Open Interest\n310 Volume ..."
24733,owfbxp,[TSLA],[70.30%],Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but..."
24734,owd2pn,[SAVA],[-42.35%],What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...


In [ ]:
# Convert ticker and growth columns to strings without square brackets
updated_df__['ticker'] = updated_df__['ticker'].apply(lambda x: ', '.join(x))
updated_df__['growth'] = updated_df__['growth'].apply(lambda x: ', '.join(x))

updated_df__

In [ ]:
data2

,id,ticker,growth,title,score,num_comments,text
8,krx451,GME,811.69%,k gme sharesoptions lets get it gme gay fam,14,42,sharesoptions k
11,ks1tzw,GME,811.69%,gme🚀🚀🚀,28,37,all right all of you autists and filthy casual...
14,ksjhuu,MGNI,47.13%,$mgni 🌈🐻 have created a wonderful buying oppor...,7,97,tldr $ pt positions $c 🚀🚀🚀🚀🚀 good afternoon re...
19,kt79b2,GME,775.88%,$gme gang ryan cohen could be attending the ic...,9,164,we might be hearing for the first time directl...
23,kt9enf,GME,775.88%,gme newbies dont lose hope this price action i...,14,8,bears feast on low volume hence why gme drops ...
...,...,...,...,...,...,...,...
3268,mfc86c,AMRS,-14.75%,$amrs 🚀 🌕 the ultimate dd 🚀 🌕 deep longterm va...,20,630,note nothing in this post is financial advice ...
3275,mk2651,PLTR,4.20%,pltr multiple news articles last weekend pltr ...,34,246,pltr palantir technologies credit suisse morga...
3276,mk0gae,VIAC,-6.18%,heres why viac is setting up for a strong rebound,10,306,viacomcbs viac is down in nearly two weeks and...
3277,mk4pod,PAVE,0.48%,an investment that will benefit from the infra...,30,40,if you are looking for an infrastructure inves...


In [ ]:
updated_df__

,id,ticker,growth,title,score,num_comments,text
0,l6uhhn,GME,-45.79%,Exit the system,0,47,The CEO of NASDAQ pushed to halt trading “to g...
1,l6uf6d,AN,44.91%,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,53,Hedgefund whales are spreading disinfo saying ...
2,l6ub9l,ARE,7.52%,THIS IS THE MOMENT,405,178,Life isn't fair. My mother always told me that...
3,l6uao1,GME,-45.79%,"We need to keep this movement going, we all ca...",222,70,I believe right now is one of those rare oppo...
4,l6u9wu,OUT,34.61%,"Once you're done with GME - $AG and $SLV, the ...",0,16,You guys are champs. GME... who would have tho...
...,...,...,...,...,...,...,...
24731,owht70,AMC,5.31%,Downtrends ... How to deal with them.,28,36,"WSB, hi. You may know me from my previous shi..."
24732,owhamt,CRSR,-14.29%,"$CRSR August 20, 2021 Unusual Options Activity 👀",200,115,"A)50$ strike\n3,048 Open Interest\n310 Volume ..."
24733,owfbxp,TSLA,70.30%,Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but..."
24734,owd2pn,SAVA,-42.35%,What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...


In [ ]:
concatenated_df = pd.concat([data2, updated_df__])

concatenated_df.reset_index(drop=True, inplace=True)

concatenated_df


,id,ticker,growth,title,score,num_comments,text
0,krx451,GME,811.69%,k gme sharesoptions lets get it gme gay fam,14,42,sharesoptions k
1,ks1tzw,GME,811.69%,gme🚀🚀🚀,28,37,all right all of you autists and filthy casual...
2,ksjhuu,MGNI,47.13%,$mgni 🌈🐻 have created a wonderful buying oppor...,7,97,tldr $ pt positions $c 🚀🚀🚀🚀🚀 good afternoon re...
3,kt79b2,GME,775.88%,$gme gang ryan cohen could be attending the ic...,9,164,we might be hearing for the first time directl...
4,kt9enf,GME,775.88%,gme newbies dont lose hope this price action i...,14,8,bears feast on low volume hence why gme drops ...
...,...,...,...,...,...,...,...
13753,owht70,AMC,5.31%,Downtrends ... How to deal with them.,28,36,"WSB, hi. You may know me from my previous shi..."
13754,owhamt,CRSR,-14.29%,"$CRSR August 20, 2021 Unusual Options Activity 👀",200,115,"A)50$ strike\n3,048 Open Interest\n310 Volume ..."
13755,owfbxp,TSLA,70.30%,Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but..."
13756,owd2pn,SAVA,-42.35%,What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...


# Preprocessing

In [ ]:
df = pd.read_csv('/content/concatenated_df.csv')

In [ ]:
df

,id,ticker,growth,title,score,num_comments,text
0,krx451,GME,811.69%,k gme sharesoptions lets get it gme gay fam,14,42,sharesoptions k
1,ks1tzw,GME,811.69%,gme🚀🚀🚀,28,37,all right all of you autists and filthy casual...
2,ksjhuu,MGNI,47.13%,$mgni 🌈🐻 have created a wonderful buying oppor...,7,97,tldr $ pt positions $c 🚀🚀🚀🚀🚀 good afternoon re...
3,kt79b2,GME,775.88%,$gme gang ryan cohen could be attending the ic...,9,164,we might be hearing for the first time directl...
4,kt9enf,GME,775.88%,gme newbies dont lose hope this price action i...,14,8,bears feast on low volume hence why gme drops ...
...,...,...,...,...,...,...,...
13753,owht70,AMC,5.31%,Downtrends ... How to deal with them.,28,36,"WSB, hi. You may know me from my previous shi..."
13754,owhamt,CRSR,-14.29%,"$CRSR August 20, 2021 Unusual Options Activity 👀",200,115,"A)50$ strike\n3,048 Open Interest\n310 Volume ..."
13755,owfbxp,TSLA,70.30%,Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but..."
13756,owd2pn,SAVA,-42.35%,What I Learned Investigating SAVA FUD Spreaders,238,87,***TLDR: Three bitter scientists partnered up ...


In [ ]:
df = df.drop_duplicates(subset='id', inplace=False)
df = df.drop_duplicates(subset='text', inplace=False)
df = df.drop_duplicates(subset='title', inplace=False)
df = df.drop('id', axis=1)

In [ ]:
df

,ticker,growth,title,score,num_comments,text
0,GME,811.69%,k gme sharesoptions lets get it gme gay fam,14,42,sharesoptions k
1,GME,811.69%,gme🚀🚀🚀,28,37,all right all of you autists and filthy casual...
2,MGNI,47.13%,$mgni 🌈🐻 have created a wonderful buying oppor...,7,97,tldr $ pt positions $c 🚀🚀🚀🚀🚀 good afternoon re...
3,GME,775.88%,$gme gang ryan cohen could be attending the ic...,9,164,we might be hearing for the first time directl...
4,GME,775.88%,gme newbies dont lose hope this price action i...,14,8,bears feast on low volume hence why gme drops ...
...,...,...,...,...,...,...
13752,HOOD,-25.53%,"Robinhood insider selling - Vlad sold 1,250,00...",5589,1098,"The title pretty much says it. Robinhood CEO, ..."
13753,AMC,5.31%,Downtrends ... How to deal with them.,28,36,"WSB, hi. You may know me from my previous shi..."
13754,CRSR,-14.29%,"$CRSR August 20, 2021 Unusual Options Activity 👀",200,115,"A)50$ strike\n3,048 Open Interest\n310 Volume ..."
13755,TSLA,70.30%,Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but..."


In [ ]:
df = df.dropna(subset=['ticker', 'growth', 'text'])


In [ ]:
df

,ticker,growth,title,score,num_comments,text
0,GME,811.69%,k gme sharesoptions lets get it gme gay fam,14,42,sharesoptions k
1,GME,811.69%,gme🚀🚀🚀,28,37,all right all of you autists and filthy casual...
2,MGNI,47.13%,$mgni 🌈🐻 have created a wonderful buying oppor...,7,97,tldr $ pt positions $c 🚀🚀🚀🚀🚀 good afternoon re...
3,GME,775.88%,$gme gang ryan cohen could be attending the ic...,9,164,we might be hearing for the first time directl...
4,GME,775.88%,gme newbies dont lose hope this price action i...,14,8,bears feast on low volume hence why gme drops ...
...,...,...,...,...,...,...
13752,HOOD,-25.53%,"Robinhood insider selling - Vlad sold 1,250,00...",5589,1098,"The title pretty much says it. Robinhood CEO, ..."
13753,AMC,5.31%,Downtrends ... How to deal with them.,28,36,"WSB, hi. You may know me from my previous shi..."
13754,CRSR,-14.29%,"$CRSR August 20, 2021 Unusual Options Activity 👀",200,115,"A)50$ strike\n3,048 Open Interest\n310 Volume ..."
13755,TSLA,70.30%,Ten Year Price Prediction for TSLA,156,204,"It’s all contingent on them mastering FSD, but..."


In [ ]:
df['text'] = df['text'].apply(lambda x: x.encode('ascii', 'ignore').decode())
df['title'] = df['title'].apply(lambda x: x.encode('ascii', 'ignore').decode())

<ipython-input-10-88e1ecbbaacf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: x.encode('ascii', 'ignore').decode())
<ipython-input-10-88e1ecbbaacf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].apply(lambda x: x.encode('ascii', 'ignore').decode())


In [ ]:
def clean_text(text):
  # remove punctuation except $
  cleaned = text.translate(str.maketrans(' ', ' ', string.punctuation.replace('$','') + '’')).lower()
  # remove links (http)
  cleaned = re.sub("http\w+", " ", cleaned)
  # remove digits
  cleaned = cleaned.translate({ord(k): None for k in string.digits})
  # remove tabs/newlines
  cleaned = cleaned.replace("\n", " ").replace("\t", " ").strip()
  # remove double space
  cleaned = re.sub(' +', ' ', cleaned)
  # remove stop words
  stop_words = set(stopwords.words('english'))
  cleaned = ' '.join(word for word in cleaned.split() if word not in stop_words).rstrip()
  return cleaned

In [ ]:
df['text'] = df['text'].apply(lambda x: clean_text(x))
df['title'] = df['title'].apply(lambda x: clean_text(x))

df['text'] = df['text'].apply(lambda x: contractions.fix(x))
df['title'] = df['title'].apply(lambda x: contractions.fix(x))

<ipython-input-12-42213816d957>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: clean_text(x))
<ipython-input-12-42213816d957>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].apply(lambda x: clean_text(x))
<ipython-input-12-42213816d957>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


def stem_words_in_string(text):

    words = text.split()

    stemmed_words = [stemmer.stem(word) for word in words]

    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text


def lemmatize_text(text):
    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = []
    for token, tag in tagged_tokens:
        if tag.startswith('V'):  # Verb
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos='v'))
        elif tag.startswith('N'):  # Noun
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos='n'))
        elif tag.startswith('J'):  # Adjective
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos='a'))
        elif tag.startswith('R'):  # Adverb
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos='r'))
        else:
            lemmatized_tokens.append(lemmatizer.lemmatize(token))
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text


In [ ]:
from nltk.metrics import edit_distance
from nltk.corpus import words

def correct_spelling(text):
    def get_closest_word(word):
        candidate_words = words.words()
        return min(candidate_words, key=lambda w: edit_distance(word, w))

    corrected_words = [get_closest_word(word) for word in text.split()]
    corrected_text = " ".join(corrected_words)
    return corrected_text

In [ ]:
df['text'] = df['text'].apply(lambda x: stem_words_in_string(x)).apply(lambda x: lemmatize_text(x))
df['title'] = df['title'].apply(lambda x: stem_words_in_string(x)).apply(lambda x: lemmatize_text(x))
# df['text'] = df['text'].apply(lambda x: correct_spelling(x))
# df['title'] = df['title'].apply(lambda x: correct_spelling(x))

<ipython-input-15-0bf90995b1b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: stem_words_in_string(x)).apply(lambda x: lemmatize_text(x))
<ipython-input-15-0bf90995b1b9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].apply(lambda x: stem_words_in_string(x)).apply(lambda x: lemmatize_text(x))


In [ ]:
# Remove '%' symbol and convert to numeric
df['growth'] = df['growth'].str.rstrip('%').astype(float)

# Describe the numerical feature
growth_stats = df['growth'].describe()

# Print the statistics
print(growth_stats)


count    13008.000000
mean        21.348149
std        120.028436
min        -88.190000
25%        -22.850000
50%          6.020000
75%         26.940000
max       8300.000000
Name: growth, dtype: float64


<ipython-input-16-0388e3115863>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['growth'] = df['growth'].str.rstrip('%').astype(float)


In [ ]:
df['label'] = df['growth'].apply(lambda x: 1 if float(x) >= 6 else 0)
df

<ipython-input-17-646b17a55104>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['growth'].apply(lambda x: 1 if float(x) >= 6 else 0)


,ticker,growth,title,score,num_comments,text,label
0,GME,811.69,k gme sharesopt let u get gme gay fam,14,42,sharesopt k,1
1,GME,811.69,gme,28,37,right autist filthi casual well listen see day...,1
2,MGNI,47.13,$ mgni creat wonder buy opportun,7,97,tldr $ pt posit $ c good afternoon retard reme...,1
3,GME,775.88,$ gme gang ryan cohen could attend icr next week,9,164,might hear first time directli ryan cohen next...,1
4,GME,775.88,gme newbi do not lose hope price action total ...,14,8,bear feast low volum henc gme drop even nasdaq...,1
...,...,...,...,...,...,...,...
13752,HOOD,-25.53,robinhood insid sell vlad sell $ million worth...,5589,1098,titl pretti much say robinhood ceo vladimir te...,0
13753,AMC,5.31,downtrend deal,28,36,wsb hi may know previou shitti gay bear post c...,0
13754,CRSR,-14.29,$ crsr august unusu option activ,200,115,a $ strike open interest volum iv larg number ...,0
13755,TSLA,70.30,ten year price predict tsla,156,204,conting master fsd win big prize loup think ge...,1


In [ ]:
df_train = df.drop('growth', axis=1)

count = df_train['label'].value_counts()
print(count)

1    6534
0    6474
Name: label, dtype: int64


In [ ]:
df_train_clean = df_train[:1304]

In [ ]:
df_train_clean

,ticker,title,score,num_comments,text,label
0,GME,k gme sharesopt let u get gme gay fam,14,42,sharesopt k,1
1,GME,gme,28,37,right autist filthi casual well listen see day...,1
2,MGNI,$ mgni creat wonder buy opportun,7,97,tldr $ pt posit $ c good afternoon retard reme...,1
3,GME,$ gme gang ryan cohen could attend icr next week,9,164,might hear first time directli ryan cohen next...,1
4,GME,gme newbi do not lose hope price action total ...,14,8,bear feast low volum henc gme drop even nasdaq...,1
...,...,...,...,...,...,...
1342,BIG,$ gme infin war updat front line belliger casu...,5004,606,financi advic boy girl listen today big day in...,1
1343,GME,fellow wsber do not sleep demo day accumul $ p...,0,32,edit alright fine last $ buy power goe $ gme o...,0
1344,GME,game theoret approach phenomenon overshort sto...,354,94,suppos fellow retard sharehold gme morph one s...,0
1345,ALL,gme war suit strike back warn fellow autist,1677,386,obligatori i be financi advisor blah blah blah...,1


# Trying Word Embedding (Glove)

In [ ]:
from tqdm import tqdm
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
embeddings_index = {}
with open('/content/glove.840B.300d.txt', encoding='utf-8') as f:
    for line_number, line in tqdm(enumerate(f, start=1)):
        line = line.replace('name@domain.com', '')  # Remove 'name@domain.com' from the line
        line = line.replace('.', '')
        line = line.replace('Killerseatscom', '')
        line = line.replace('mylotcom', '')
        line = line.replace('Amazoncom', '')        # Remove all occurrences of periods

        line = line.strip()  # Remove leading/trailing whitespaces

        # if not line:  # Skip empty lines
        #     continue

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except ValueError as e:
            print(f"Error in line {line_number}: {line}")
            print(f"Error message: {str(e)}")
            continue
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))




2196017it [03:17, 11136.71it/s]

Found 2134042 word vectors.


In [ ]:
def sent2vec(s):
    words = str(s).lower().split()
    words = [w for w in words if w not in stop_words]
    words = [w for w in words if w.isalpha()]

    vectors = []
    for w in words:
        if w in embeddings_index:
            vectors.append(embeddings_index[w])
    else:
        # Handle the case when word is not found in embeddings
        vectors.append(np.zeros(300, dtype=np.float32))  # Replace with appropriate size

    M = np.array(vectors, dtype=np.float32)
    v = np.sum(M, axis=0)
    if np.count_nonzero(v) == 0:
        return np.zeros(300, dtype=np.float32)
    return v / np.sqrt(np.sum(v ** 2))

In [ ]:
df_train_clean['text'] = df_train_clean['title'] + ' ' + df_train_clean['text']

<ipython-input-84-ca1556e23563>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_clean['text'] = df_train_clean['title'] + ' ' + df_train_clean['text']


In [ ]:
df_train_clean = df_train_clean.drop('title', axis=1)

In [ ]:
df_train_clean

,ticker,score,num_comments,text,label
0,GME,14,42,k gme sharesopt let u get gme gay fam k gme sh...,1
1,GME,28,37,gme gme gme gme right autist filthi casual wel...,1
2,MGNI,7,97,$ mgni creat wonder buy opportun $ mgni creat ...,1
3,GME,9,164,$ gme gang ryan cohen could attend icr next we...,1
4,GME,14,8,gme newbi do not lose hope price action total ...,1
...,...,...,...,...,...
1342,BIG,5004,606,$ gme infin war updat front line belliger casu...,1
1343,GME,0,32,fellow wsber do not sleep demo day accumul $ p...,0
1344,GME,354,94,game theoret approach phenomenon overshort sto...,0
1345,ALL,1677,386,gme war suit strike back warn fellow autist gm...,1


In [ ]:
df_train_clean_other = df_train_clean.drop('text', axis=1).values

In [ ]:
df_train_clean_text = df_train_clean['text']

In [ ]:
df_train_clean_text = df_train_clean_text.values

In [ ]:
df_train_clean_text

array(['k gme sharesopt let u get gme gay fam k gme sharesopt let u get gme gay fam k gme sharesopt let u get gme gay fam k gme sharesopt let u get gme gay fam sharesopt k',
       'gme gme gme gme right autist filthi casual well listen see day day diamond squad gay bear heat battl gay bear label gme trash compani want shop video game actual store suckl straight daddi bezo tit diamond squad certain ride back ryan cohen great squeez akin way babi sloth hang onto mother bear convinc squeez imposs melvin band cuck liter wayyyyy smart self proclaim autist know get cover short make bank retard like fun eat ramen noodl warm piss overpass unshaken diamond squad hold tight thing know hold tightli onto back mama trust bring high canopi here be thing squeez squoz go uranu next tuesday squeez come here be melvin band cuck go keep short one two thing happen gme goe busi mama cohen slowli climb secret rocket gift guess papa elon ryan cohen take compani turn goldmin cuck seriou troubl longer short a

In [ ]:
df_train_clean_other

array([['GME', 14, 42, 1],
       ['GME', 28, 37, 1],
       ['MGNI', 7, 97, 1],
       ...,
       ['GME', 354, 94, 0],
       ['ALL', 1677, 386, 1],
       ['AMC', 691, 181, 1]], dtype=object)

In [ ]:
df_train_clean_text_glove = np.array([sent2vec(x) for x in tqdm(df_train_clean_text)])

# Concatenate text feature embeddings with other features
# train_glove = np.concatenate((df_train_clean_text_glove, df_train_clean_other), axis=1)

100%|██████████| 1304/1304 [00:01<00:00, 928.10it/s]


In [ ]:
df_train_clean_text_glove

array([[-0.04582318, -0.01295356,  0.02266255, ...,  0.04765758,
        -0.0516178 ,  0.00560522],
       [ 0.04582635,  0.01113418,  0.02889102, ..., -0.00661397,
        -0.00501551, -0.01425136],
       [ 0.05858121,  0.02924729,  0.00454181, ...,  0.00768698,
        -0.06233425, -0.00816102],
       ...,
       [ 0.05020397,  0.02373591,  0.05373418, ..., -0.04153134,
        -0.00135679, -0.03149295],
       [ 0.05740876,  0.01339545,  0.0607869 , ...,  0.01145167,
        -0.00470839, -0.02492956],
       [ 0.04719305,  0.03084964,  0.07505569, ..., -0.04818753,
        -0.01275668, -0.01071204]], dtype=float32)

In [ ]:
df_train_clean_other

array([['GME', 14, 42, 1],
       ['GME', 28, 37, 1],
       ['MGNI', 7, 97, 1],
       ...,
       ['GME', 354, 94, 0],
       ['ALL', 1677, 386, 1],
       ['AMC', 691, 181, 1]], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Label encode the ticker column in the other features array
le = LabelEncoder()
other_features_encoded = df_train_clean_other.copy()
other_features_encoded[:, 0] = le.fit_transform(df_train_clean_other[:, 0])

# Concatenate the GloVe array and the other features array
df_glove = pd.DataFrame(df_train_clean_text_glove, columns=[f'glove_{i}' for i in range(df_train_clean_text_glove.shape[1])])
df_other = pd.DataFrame(other_features_encoded, columns=['ticker', 'score', 'num_comments', 'label'])


In [ ]:
df_other = df_other.astype(np.float32)

In [ ]:
df_other.values

array([[5.200e+01, 1.400e+01, 4.200e+01, 1.000e+00],
       [5.200e+01, 2.800e+01, 3.700e+01, 1.000e+00],
       [7.700e+01, 7.000e+00, 9.700e+01, 1.000e+00],
       ...,
       [5.200e+01, 3.540e+02, 9.400e+01, 0.000e+00],
       [4.000e+00, 1.677e+03, 3.860e+02, 1.000e+00],
       [6.000e+00, 6.910e+02, 1.810e+02, 1.000e+00]], dtype=float32)

In [ ]:
# df_other["ticker"] = pd.to_numeric(df_other["ticker"], errors="coerce")
# df_other["score"] = pd.to_numeric(df_other["score"], errors="coerce")
# df_other["num_comments"] = pd.to_numeric(df_other["num_comments"], errors="coerce")
# df_other["label"] = pd.to_numeric(df_other["label"], errors="coerce")

In [ ]:
df_concatenated = pd.concat([df_glove, df_other], axis=1)

# Convert ticker column back to original labels using inverse_transform
# df_concatenated['ticker'] = le.inverse_transform(df_concatenated['ticker'])

# Print the concatenated DataFrame


In [ ]:
df_concatenated

,glove_0,glove_1,glove_2,glove_3,glove_4,glove_5,glove_6,glove_7,glove_8,glove_9,...,glove_294,glove_295,glove_296,glove_297,glove_298,glove_299,ticker,score,num_comments,label
0,-0.045823,-0.012954,0.022663,-0.068381,0.030454,0.065030,-0.070882,-0.083306,0.041684,0.000334,...,0.142427,0.051180,0.042075,0.047658,-0.051618,0.005605,52.0,14.0,42.0,1.0
1,0.045826,0.011134,0.028891,-0.032023,0.003694,-0.014637,-0.000923,-0.006131,-0.037231,-0.051855,...,0.080933,0.022632,0.043941,-0.006614,-0.005016,-0.014251,52.0,28.0,37.0,1.0
2,0.058581,0.029247,0.004542,-0.007494,-0.016524,-0.078254,-0.003755,0.042539,0.005159,-0.075204,...,0.089488,-0.001343,0.045273,0.007687,-0.062334,-0.008161,77.0,7.0,97.0,1.0
3,-0.005926,-0.007322,0.035389,0.008420,-0.031473,-0.067426,0.035189,-0.024860,-0.021757,-0.019212,...,0.128686,-0.011224,0.059307,-0.007597,-0.039480,-0.073550,52.0,9.0,164.0,1.0
4,0.048797,0.030188,0.065641,-0.002759,-0.020351,-0.018712,0.006515,0.006002,-0.040173,-0.040765,...,0.057176,0.043009,0.006339,0.015799,0.000503,-0.057553,52.0,14.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,0.055131,-0.001905,0.036548,-0.085352,-0.031842,-0.002383,0.013283,0.018927,-0.046261,-0.045841,...,0.085860,0.014952,0.012269,0.032420,-0.044594,-0.030716,22.0,5004.0,606.0,1.0
1300,0.051654,0.027827,0.034469,-0.037894,-0.018234,-0.043275,-0.008404,0.019660,-0.051967,-0.060417,...,0.068565,0.005270,0.000480,0.038226,-0.050043,-0.022695,52.0,0.0,32.0,0.0
1301,0.050204,0.023736,0.053734,-0.058595,0.004393,-0.023637,-0.025288,0.027441,-0.038956,-0.061243,...,0.059635,0.006349,0.010880,-0.041531,-0.001357,-0.031493,52.0,354.0,94.0,0.0
1302,0.057409,0.013395,0.060787,-0.085678,-0.041185,-0.018977,-0.010602,0.037758,-0.050176,-0.067149,...,0.065856,0.010670,0.009957,0.011452,-0.004708,-0.024930,4.0,1677.0,386.0,1.0


In [ ]:
X = df_concatenated.drop('label', axis=1)
y = df_concatenated['label']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def check_alphabets(df):
    for column in df.columns:
        if df[column].apply(lambda x: any(c.isalpha() for c in str(x))).any():
            print(f"Alphabets found in column '{column}'")
        else:
            print(f"No alphabets found in column '{column}'")

In [ ]:
check_alphabets(X_train)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

lr_model = LogisticRegression()


clf = Pipeline([
    ('lr', lr_model)
])

param_grid = {
    'lr__C': [0.1, 1.0, 10],
    'lr__penalty': ['l1'],
    'lr__solver': ['liblinear'],
    'lr__max_iter': [100, 400, 500],
}

model = GridSearchCV(estimator=clf, param_grid=param_grid,  scoring='accuracy', verbose=10, n_jobs=-1, cv=2)


model.fit(X_train, y_train)

best_params = model.best_params_
best_score = model.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
Best Parameters: {'lr__C': 1.0, 'lr__max_iter': 100, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
Best Score: 0.6318382715232276


In [ ]:
'''Best Parameters: {'lr__C': 1.0, 'lr__max_iter': 100, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
Best Score: 0.6318382715232276'''

In [ ]:
lr_model = LogisticRegression()

clf = Pipeline([
    ('lr', lr_model)
])

param_grid = {
    'lr__C': [0.1, 1.0, 10],
    'lr__penalty': ['l2'],
    'lr__solver': ['lbfgs'],
    'lr__max_iter': [100, 400, 500],
}

model = GridSearchCV(estimator=clf, param_grid=param_grid,  scoring='accuracy', verbose=10, n_jobs=-1, cv=2)


model.fit(X_train, y_train)

best_params = model.best_params_
best_score = model.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
Best Parameters: {'lr__C': 1.0, 'lr__max_iter': 100, 'lr__penalty': 'l2', 'lr__solver': 'lbfgs'}
Best Score: 0.6222725969069208


In [ ]:
'''Best Parameters: {'lr__C': 1.0, 'lr__max_iter': 100, 'lr__penalty': 'l2', 'lr__solver': 'lbfgs'}
Best Score: 0.6222725969069208'''

In [ ]:
xgb_clf = xgb.XGBClassifier(max_depth=6, n_estimators=200, colsample_bytree=1.0,
                        subsample=0.5, nthread=10, learning_rate=0.01)

xgb_clf.fit(X_train, y_train)

y_pred = xgb_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7279693486590039


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

svc_model = svm.LinearSVC()

clf = Pipeline([
    ('svc', svc_model)
])

param_grid = {
    'svc__C': [0.1, 1.0, 10.0],
    'svc__penalty': ['l1', 'l2'],
    'svc__loss': ['squared_hinge'],
    'svc__dual': [False],
    'svc__class_weight': [None, 'balanced'],
    'svc__max_iter': [1000, 2000]
}



grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, verbose=10, n_jobs=-1, cv=3)

grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best parameters:  {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__dual': False, 'svc__loss': 'squared_hinge', 'svc__max_iter': 1000, 'svc__penalty': 'l2'}
Best score:  0.6567347930261574


In [ ]:
'''Best parameters:  {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__dual': False, 'svc__loss': 'squared_hinge', 'svc__max_iter': 1000, 'svc__penalty': 'l2'}
Best score:  0.6567347930261574'''

In [ ]:
nan_count = df_concatenated.isna().sum().sum()

print("Number of NaN values:", nan_count)

Number of NaN values: 0


# Trying TfidfVectorizer

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse


In [ ]:
one_hot_encoder = OneHotEncoder()
ticker_encoded = one_hot_encoder.fit_transform(df_train_clean[['ticker']]).toarray()
ticker_columns = one_hot_encoder.get_feature_names_out()

df_encoded = pd.DataFrame(data=ticker_encoded, columns=ticker_columns)

In [ ]:
df_encoded_imputer = SimpleImputer(strategy='median')
df_encoded_imputer.fit(df_encoded)

SimpleImputer(strategy='median')

In [ ]:
corpus = df_train_clean["title"].astype(str) + " " + df_train_clean["text"].astype(str)


tfv = TfidfVectorizer(min_df=3, max_features=None,
                      strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
                      ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
                      stop_words='english')

tfidf_data = tfv.fit_transform(corpus)


feature_names = tfv.get_feature_names_out()


tfidf_df = pd.DataFrame(data=tfidf_data.toarray(), columns=feature_names)



/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


In [ ]:
tfidf_df_imputer = SimpleImputer(strategy='median')
df_encoded_imputer.fit(tfidf_df)

SimpleImputer(strategy='median')

In [ ]:
tfidf_sparse = sparse.csr_matrix(tfidf_df.values)

In [ ]:
tfidf_sparse = tfidf_sparse.astype('float32')

In [ ]:
import sys
import scipy.sparse as sp


In [ ]:
print(tfidf_sparse.shape)
print(tfidf_sparse.nnz)
print(tfidf_sparse.dtype)
print(tfidf_sparse.data)
print(tfidf_sparse.indices)
print(tfidf_sparse.indptr)



memory_usage_bytes = tfidf_sparse.data.nbytes

memory_usage_mb = memory_usage_bytes / (1024 * 1024)

print("Memory Usage:", memory_usage_mb, "MB")

(1304, 9049)
111386
float32
[0.39123857 0.17166357 0.40087166 ... 0.09153883 0.05833962 0.05901292]
[2917 2950 3972 ... 8944 8947 9029]
[     0      9    135 ... 110981 111234 111386]
Memory Usage: 0.42490386962890625 MB


In [ ]:
tfidf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Columns: 9049 entries, aal to zoom
dtypes: float64(9049)
memory usage: 90.0 MB


In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

In [ ]:
imputer.fit(combined_df)


SimpleImputer(strategy='median')

In [ ]:
tfidf_sparse.shape

(1304, 9049)

In [ ]:
df_encoded_sparse = sparse.csr_matrix(df_encoded.values)

In [ ]:
df_train_sparse = sparse.csr_matrix(df_train_clean[['label', 'score', 'num_comments']].values)

In [ ]:
df_train_sparse

<1304x3 sparse matrix of type '<class 'numpy.int64'>'
	with 3230 stored elements in Compressed Sparse Row format>

In [ ]:
df_encoded_sparse

<1304x146 sparse matrix of type '<class 'numpy.float64'>'
	with 1304 stored elements in Compressed Sparse Row format>

In [ ]:
combined_sparse = sp.hstack((df_train_sparse, df_encoded_sparse, tfidf_sparse))

combined_df = pd.DataFrame.sparse.from_spmatrix(combined_sparse)

In [ ]:
combined_df

,0,1,2,3,4,5,6,7,8,9,...,9188,9189,9190,9191,9192,9193,9194,9195,9196,9197
0,1.0,14.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,1.0,28.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,1.0,7.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,1.0,9.0,164.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,1.0,14.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1299,1.0,5004.0,606.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1300,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1301,0.0,354.0,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068196,0.0,0.0
1302,1.0,1677.0,386.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
X = combined_df.drop(0, axis=1)
y = combined_df[0]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model_m = MultinomialNB()
model_m.fit(X_train, y_train)

MultinomialNB()

In [ ]:
model_l1 = LogisticRegression(penalty='l1', solver='liblinear')

model_l1.fit(X_train, y_train)

y_pred_l1 = model_l1.predict(X_test)

accuracy_l1 = accuracy_score(y_test, y_pred_l1)
print("Accuracy (L1):", accuracy_l1)

Accuracy (L1): 0.7126436781609196


In [ ]:
model_l2 = LogisticRegression(penalty='l2')

model_l2.fit(X_train, y_train)

y_pred_l2 = model_l2.predict(X_test)

accuracy_l2 = accuracy_score(y_test, y_pred_l2)
print("Accuracy (L2):", accuracy_l2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy (L2): 0.6819923371647509


In [ ]:
y_pred = model_m.predict(X_test)


accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
f1_score = metrics.f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1_score)


Accuracy: 0.6398467432950191
Precision: 0.6376811594202898
Recall: 0.6666666666666666
F1-Score: 0.6518518518518518


In [ ]:
print(X_train.dtypes)


1       Sparse[float64, 0]
2       Sparse[float64, 0]
3       Sparse[float64, 0]
4       Sparse[float64, 0]
5       Sparse[float64, 0]
               ...        
9193    Sparse[float64, 0]
9194    Sparse[float64, 0]
9195    Sparse[float64, 0]
9196    Sparse[float64, 0]
9197    Sparse[float64, 0]
Length: 9197, dtype: object


In [ ]:
print(y_train.dtypes)


Sparse[float64, 0]


In [ ]:
from sklearn import svm

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

svc_model = svm.LinearSVC(C=1.0, max_iter=10000, tol=0.0001)

svc_model.fit(X_train, y_train)

y_pred = svc_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy: 0.7624521072796935


In [ ]:
import pickle

In [ ]:
with open('linearsvc_model_76.pkl', 'wb') as file:
    pickle.dump(svc_model, file)


In [ ]:
with open('linearsvc_model_76.pkl', 'rb') as file:
    svc_model = pickle.load(file)




In [ ]:


def preprocess(random_row):

  random_row['text'] = random_row["title"].astype(str) + " " + random_row["text"].astype(str)
  random_row = random_row.drop('title', axis=1)


  random_row['text'] = random_row['text'].apply(lambda x: x.encode('ascii', 'ignore').decode())
  random_row['text'] = random_row['text'].apply(lambda x: clean_text(x))
  random_row['text'] = random_row['text'].apply(lambda x: contractions.fix(x))
  random_row['text'] = random_row['text'].apply(lambda x: stem_words_in_string(x)).apply(lambda x: lemmatize_text(x))





  random_row_encoded = one_hot_encoder.transform(random_row[['ticker']]).toarray()
  random_row_tfidf = tfv.transform(random_row['text'].astype(str))

  random_row_encoded_df = pd.DataFrame(data=random_row_encoded, columns=ticker_columns)
  random_row_tfidf_df = pd.DataFrame(data=random_row_tfidf.toarray(), columns=feature_names)

  random_row_encoded_df.dropna(0)
  random_row_tfidf_df.dropna(0)

  random_row_encoded_df_sparse = sparse.csr_matrix(random_row_encoded_df.values)
  random_row_tfidf_df_sparse = sparse.csr_matrix(random_row_tfidf_df.values)

  random_row_sparse = sparse.csr_matrix(random_row[['score', 'num_comments']].values)

  combined_sparse = sp.hstack((random_row_sparse, random_row_encoded_df_sparse, random_row_tfidf_df_sparse))

  random_row_features = pd.DataFrame.sparse.from_spmatrix(combined_sparse)

  random_row_features = random_row_features[:1]




  actual_label = random_row['label']
  actual_label = pd.DataFrame(actual_label)

  actual_label_sparse = sparse.csr_matrix(actual_label.values)

  actual_label = pd.DataFrame.sparse.from_spmatrix(actual_label_sparse)


  random_row_features.dropna(0)



  random_row_index = random_row_features.index[0]
  random_row_features.loc[random_row_index].fillna(0, inplace=True)


  print(random_row_features.dtypes)
  print(actual_label.dtypes)

  df_encoded_sparse = sparse.csr_matrix(df_encoded.values)



  return random_row_features, actual_label


In [ ]:
import random

random_row = df_train_clean.sample(n=1)
print(random_row.values)
print(random_row['label'].values[0])

nan_count = random_row.isna().sum().sum()

print("Number of NaN values:", nan_count)

[['IPOE' 'lord chamath ipoesofi' 11 18
  'give retard head armi tard readi pump lord chamath spac' 0]]
0
Number of NaN values: 0


In [ ]:
X, y = preprocess(random_row)

<ipython-input-106-d525985cce73>:22: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  random_row_encoded_df.dropna(0)
<ipython-input-106-d525985cce73>:23: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  random_row_tfidf_df.dropna(0)
<ipython-input-106-d525985cce73>:47: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  random_row_features.dropna(0)


0       Sparse[float64, 0]
1       Sparse[float64, 0]
2       Sparse[float64, 0]
3       Sparse[float64, 0]
4       Sparse[float64, 0]
               ...        
9192    Sparse[float64, 0]
9193    Sparse[float64, 0]
9194    Sparse[float64, 0]
9195    Sparse[float64, 0]
9196    Sparse[float64, 0]
Length: 9197, dtype: object
0    Sparse[int64, 0]
dtype: object


<ipython-input-106-d525985cce73>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_row_features.loc[random_row_index].fillna(0, inplace=True)


In [ ]:
prediction = svc_model.predict(X)
print('Predicted label:', prediction)
print('Actual label:', y)

Predicted label: [0.]
Actual label:    0
0  0


In [ ]:
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [ ]:
X_train_sparse = sparse.csr_matrix(X_train.values)
X_test_sparse = sparse.csr_matrix(X_test.values)

In [ ]:
xgb_clf = xgb.XGBClassifier(max_depth=6, n_estimators=200, colsample_bytree=0.8, subsample=0.8)

xgb_clf.fit(X_train_sparse.tocsc(), y_train)

y_pred = xgb_clf.predict(X_test_sparse.tocsc())

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

/usr/local/lib/python3.10/dist-packages/xgboost/data.py:438: FutureWarning: The behavior of .astype from SparseDtype to a non-sparse dtype is deprecated. In a future version, this will return a non-sparse array with the requested dtype. To retain the old behavior, use `obj.astype(SparseDtype(dtype))`
  data = data.values.astype('float')


Accuracy: 0.7241379310344828


In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


lr_model = LogisticRegression()

clf = Pipeline([
    ('lr', lr_model)
])

param_grid = {
    'lr__C': [0.1, 1.0, 10],
    'lr__penalty': ['l1'],
    'lr__solver': ['liblinear'],
    'lr__max_iter': [100, 400, 500],
}

model = GridSearchCV(estimator=clf, param_grid=param_grid,  scoring='accuracy', verbose=10, n_jobs=-1, cv=2)


model.fit(X_train, y_train)

best_params = model.best_params_
best_score = model.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
Best Parameters: {'lr__C': 1.0, 'lr__max_iter': 100, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
Best Score: 0.6864598730705025


In [ ]:
'''Best Parameters: {'lr__C': 1.0, 'lr__max_iter': 100, 'lr__penalty': 'l1', 'lr__solver': 'liblinear'}
Best Score: 0.6864598730705025 '''

In [ ]:
lr_model = LogisticRegression()

clf = Pipeline([
    ('lr', lr_model)
])

param_grid = {
    'lr__C': [0.1, 1.0, 10],
    'lr__penalty': ['l2'],
    'lr__solver': ['lbfgs'],
    'lr__max_iter': [100, 400, 500],
}

model = GridSearchCV(estimator=clf, param_grid=param_grid,  scoring='accuracy', verbose=10, n_jobs=-1, cv=2)


model.fit(X_train, y_train)

best_params = model.best_params_
best_score = model.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
Best Parameters: {'lr__C': 10, 'lr__max_iter': 400, 'lr__penalty': 'l2', 'lr__solver': 'lbfgs'}
Best Score: 0.7027636213882822


In [ ]:
'''Best Parameters: {'lr__C': 10, 'lr__max_iter': 400, 'lr__penalty': 'l2', 'lr__solver': 'lbfgs'}
Best Score: 0.7027636213882822'''

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

xgb_model = xgb.XGBClassifier()

clf = Pipeline([
    ('xgb', xgb_model)
])

param_grid = {
    'xgb__max_depth': [6, 10],
    'xgb__n_estimators': [100, 200],
    'xgb__learning_rate': [0.1, 0.01],
    'xgb__colsample_bytree': [0.8, 1.0],
    'xgb__subsample': [0.8, 1.0],
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, verbose=10, n_jobs=-1, cv=2)

grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 32 candidates, totalling 64 fits


/usr/local/lib/python3.10/dist-packages/xgboost/data.py:438: FutureWarning: The behavior of .astype from SparseDtype to a non-sparse dtype is deprecated. In a future version, this will return a non-sparse array with the requested dtype. To retain the old behavior, use `obj.astype(SparseDtype(dtype))`
  data = data.values.astype('float')


Best parameters:  {'xgb__colsample_bytree': 0.8, 'xgb__learning_rate': 0.01, 'xgb__max_depth': 10, 'xgb__n_estimators': 100, 'xgb__subsample': 0.8}
Best score:  0.6951173325685206
	xgb__colsample_bytree: 0.8
	xgb__learning_rate: 0.01
	xgb__max_depth: 10
	xgb__n_estimators: 100
	xgb__subsample: 0.8


In [ ]:
''' Best parameters:  {'xgb__colsample_bytree': 0.8, 'xgb__learning_rate': 0.01, 'xgb__max_depth': 10, 'xgb__n_estimators': 100, 'xgb__subsample': 0.8}
Best score:  0.6951173325685206
	xgb__colsample_bytree: 0.8
	xgb__learning_rate: 0.01
	xgb__max_depth: 10
	xgb__n_estimators: 100
	xgb__subsample: 0.8 '''


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('nb', MultinomialNB())
])

param_grid = {
    'nb__alpha': [0.1, 1.0, 10.0],
    'nb__fit_prior': [True, False],
    'nb__class_prior': [None, [0.4, 0.6]]
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=3)

grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Best parameters:  {'nb__alpha': 0.1, 'nb__class_prior': [0.4, 0.6], 'nb__fit_prior': True}
Best score:  0.6490195104177018


In [ ]:
'''Best parameters:  {'nb__alpha': 0.1, 'nb__class_prior': [0.4, 0.6], 'nb__fit_prior': True}
Best score:  0.6490195104177018   '''

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

svc_model = svm.LinearSVC()

clf = Pipeline([
    ('svc', svc_model)
])

param_grid = {
    'svc__C': [0.1, 1.0, 10.0],
    'svc__penalty': ['l1', 'l2'],
    'svc__loss': ['squared_hinge'],
    'svc__dual': [False],
    'svc__class_weight': [None, 'balanced'],
    'svc__max_iter': [1000, 2000]
}



grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, verbose=10, n_jobs=-1, cv=3)

grid_search.fit(X_train_sparse.tocsc(), y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best parameters:  {'svc__C': 1.0, 'svc__class_weight': None, 'svc__dual': False, 'svc__loss': 'squared_hinge', 'svc__max_iter': 1000, 'svc__penalty': 'l2'}
Best score:  0.7200304746762066


In [ ]:
'''
Best parameters:  {'svc__C': 1.0, 'svc__class_weight': None, 'svc__dual': False, 'svc__loss': 'squared_hinge', 'svc__max_iter': 1000, 'svc__penalty': 'l2'}
Best score:  0.7200304746762066'''